In [13]:
import pandas as pd
# from utils import add_prefix_to_colnames, percentile, onehot, summary_group_cate_data, summary_group_num_data

# Create after_trans_payment_for_loan.csv
For cost calculation

In [14]:
loan_df = pd.read_csv('data/loan.csv')
trans_before_loan = pd.read_csv('data/trans_before_loan.csv')
trans_after_loan = pd.read_csv('data/trans_after_loan.csv')

Find out that Monthly transaction of after transactin, when stop to pay loan.

In [15]:
trans_df_in = trans_after_loan[trans_after_loan['k_symbol'].notna()]
trans_df_in.head()

,account_id,date,type,operation,amount,balance,k_symbol,amount_in,amount_out,loan_date,day_after_loan
0,2,1994-01-05,withdrawal,remittance_to_anotherbank,7266,20575,household,0.0,7266.0,1994-01-05,0.0
2,2,1994-01-12,withdrawal,remittance_to_anotherbank,3373,35338,loan,0.0,3373.0,1994-01-05,-7.0
8,2,1994-01-31,withdrawal,withdrawal_in_cash,15,20039,payment_for_statement,0.0,15.0,1994-01-05,-26.0
9,2,1994-01-31,credit,NaN,115,20053,interest_credited,115.0,0.0,1994-01-05,-26.0
10,2,1994-02-05,withdrawal,remittance_to_anotherbank,7266,12773,household,0.0,7266.0,1994-01-05,-31.0


In [16]:
trans_df_ = trans_df_in[trans_df_in['k_symbol'] == 'loan'].sort_values(by=['account_id', 'date']).copy()
trans_df_ = trans_df_.groupby(by='account_id')[['k_symbol']].count().reset_index()
trans_df_ = trans_df_.rename(columns={'k_symbol':'count_loan_trans'})
trans_df_

,account_id,count_loan_trans
0,2,24
1,19,8
2,25,12
3,37,1
4,38,8
...,...,...
677,11327,3
678,11328,5
679,11349,38
680,11359,12


In [17]:
df = pd.merge(loan_df, trans_df_, how='left', on=['account_id'])
df.head()

,loan_id,account_id,date,amount,duration,payments,status,count_loan_trans
0,4959,2,1994-01-05,80952,24,3373.0,A,24
1,4961,19,1996-04-29,30276,12,2523.0,B,8
2,4962,25,1997-12-08,30276,12,2523.0,A,12
3,4967,37,1998-10-14,318480,60,5308.0,D,1
4,4968,38,1998-04-19,110736,48,2307.0,C,8


In [18]:
df.loc[df['status']=='C', ['count_loan_trans']] = df['duration']
df.head()

,loan_id,account_id,date,amount,duration,payments,status,count_loan_trans
0,4959,2,1994-01-05,80952,24,3373.0,A,24
1,4961,19,1996-04-29,30276,12,2523.0,B,8
2,4962,25,1997-12-08,30276,12,2523.0,A,12
3,4967,37,1998-10-14,318480,60,5308.0,D,1
4,4968,38,1998-04-19,110736,48,2307.0,C,48


In [19]:
t = df.loc[df['status']=='B']
# t['num_month_paid'] = t['duration'] - t['count_loan_trans']
t['%num_month_paid'] = t['count_loan_trans'].div( t['duration'], axis=0) * 100
t['%num_month_paid'].mean()

C:\Users\Sorayut Meeyim\AppData\Local\Temp\ipykernel_15320\768220784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['%num_month_paid'] = t['count_loan_trans'].div( t['duration'], axis=0) * 100


77.44623655913978

assume that D paid for 80 percent of total loan amount

In [20]:
df.loc[df['status']=='D', ['count_loan_trans']] = (df['duration'] * 80/100).round()
df = df[['account_id', 'count_loan_trans']]
df

,account_id,count_loan_trans
0,2,24.0
1,19,8.0
2,25,12.0
3,37,48.0
4,38,48.0
...,...,...
677,11327,24.0
678,11328,60.0
679,11349,60.0
680,11359,12.0


In [21]:
df.to_csv('report/after_trans_payment_for_loan.csv', index=False)

# Profit calculation

In [22]:
import numpy as np
import pandas as pd
from profit import cal_cost, cal_revenue

In [23]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

def profit_thresholding(df, th_, profit, interest_rate):
    df['y_pred'] = to_labels(df['probs'], th_)
    # TN
    df.loc[(df['y_true']==0) & (df['y_pred']==0), ['profit']] = profit
    df.loc[(df['y_true']==0) & (df['y_pred']==0), ['conf']] = 'tn'

    # TP 
    df.loc[(df['y_true']==1) & (df['y_pred']==1), ['profit']] = 0
    df.loc[(df['y_true']==1) & (df['y_pred']==1), ['conf']] = 'tp'

    # FN
    df.loc[(df['y_true']==0) & (df['y_pred']==1), ['profit']] = 0
    df.loc[(df['y_true']==0) & (df['y_pred']==1), ['conf']] = 'fn'
    
    # FP
    df.loc[(df['y_true']==1) & (df['y_pred']==0), ['profit']] = profit
    df.loc[(df['y_true']==1) & (df['y_pred']==0), ['conf']] = 'fp'
            
    df['threshold'] = th_
    df['interest_rate'] = interest_rate
    
    return df

In [25]:
report = pd.read_csv('report/report_xgb.csv') # from model.ipynb
report['y_pred'] = np.nan
report['case_TP'] = np.nan
report['case_FN'] = np.nan
report['case_FP'] = np.nan
report['case_TN'] = np.nan
report['threshold'] = np.nan
report

,y_true,probs,account_id,y_pred,case_TP,case_FN,case_FP,case_TN,threshold
0,0.0,0.017466,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.639199,19.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.030918,25.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.992415,37.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.054533,38.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
677,0.0,0.009702,11327.0,NaN,NaN,NaN,NaN,NaN,NaN
678,0.0,0.053159,11328.0,NaN,NaN,NaN,NaN,NaN,NaN
679,0.0,0.362870,11349.0,NaN,NaN,NaN,NaN,NaN,NaN
680,0.0,0.213898,11359.0,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
loan_df = pd.read_csv('data/loan.csv')
loan_df.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C


In [28]:
filename = 'report/after_trans_payment_for_loan.csv'
after_trans_for_loan = pd.read_csv(filename)

interest_rate_peryear = [0.01, 0.03, 0.05, 0.1, 0.2]
profits = []
report = pd.read_csv('report/report_xgb.csv')
reports = []

for i in interest_rate_peryear:
    revenue = cal_revenue(total_amount=loan_df['amount'], 
                    loan_duration_month=loan_df['duration'], 
                    paid_month=after_trans_for_loan['count_loan_trans'], 
                    interest_rate_peryear=i)

    cost = cal_cost(loan_payment=loan_df['payments'], 
                    loan_duration_month=loan_df['duration'],
                    paid_month=after_trans_for_loan['count_loan_trans'])

    profit = revenue - cost
    
    for th in np.arange(0, 1.1, 0.1):
        th_ = th.round(decimals=1)
        r = profit_thresholding(report, th_, profit, i).reset_index(drop=True)
        reports.append(r)
        
    profits.append(profit.sum())

reports = pd.concat(reports)
reports

,y_true,probs,account_id,y_pred,profit,conf,threshold,interest_rate
0,0.0,0.017466,2.0,1,0.000000,fn,0.0,0.01
1,1.0,0.639199,19.0,1,0.000000,tp,0.0,0.01
2,0.0,0.030918,25.0,1,0.000000,fn,0.0,0.01
3,1.0,0.992415,37.0,1,0.000000,tp,0.0,0.01
4,0.0,0.054533,38.0,1,0.000000,fn,0.0,0.01
...,...,...,...,...,...,...,...,...
677,0.0,0.009702,11327.0,0,11968.000000,tn,1.0,0.20
678,0.0,0.053159,11328.0,0,167737.453704,tn,1.0,0.20
679,0.0,0.362870,11349.0,0,251139.645062,tn,1.0,0.20
680,0.0,0.213898,11359.0,0,9004.000000,tn,1.0,0.20


In [29]:
reports.to_csv('report/report_xgb_threshold_profit.csv', index=False)

In [30]:
ori_profit = pd.DataFrame({'interest rate per year': interest_rate_peryear, 
              'Old profit':profits})

ori_profit.to_csv('report/ori_profit.csv', index=False)